## reference
- https://www.attrs.org/en/stable/

- => Use standard package 'dataclasses' from python 3.7

In [1]:
from attr import asdict, define, make_class, Factory

In [2]:
@define
class MyClass:
    a_number: int = 42
    list_of_numbers: list[int] = Factory(list)
    
    def hard_math(self, another_number):
        return self.a_number + sum(self.list_of_numbers) * another_number
    


In [3]:
mc = MyClass(5, [1,2,3,4])
mc

MyClass(a_number=5, list_of_numbers=[1, 2, 3, 4])

In [4]:
mc.hard_math(5)

55

In [5]:
mc == MyClass(5, [1,2,3,4])

True

In [6]:
asdict(mc)

{'a_number': 5, 'list_of_numbers': [1, 2, 3, 4]}

In [7]:
MyClass()


MyClass(a_number=42, list_of_numbers=[])

In [8]:
SampleClass = make_class("SampleClass", ["a", "b"])
SampleClass('foo', 'bar')


SampleClass(a='foo', b='bar')

In [9]:
asdict(SampleClass('John', 'McCartney'))

{'a': 'John', 'b': 'McCartney'}

## @dataclass
- https://qiita.com/tag1216/items/13b032348c893667862a


### dataclasses_jsonschema
- https://pypi.org/project/dataclasses-jsonschema/
  - https://github.com/s-knibbs/dataclasses-jsonschema/blob/master/tests/conftest.py

In [10]:
from dataclasses import dataclass
from dataclasses_jsonschema import JsonSchemaMixin

In [11]:
@dataclass
class Point(JsonSchemaMixin):
    """a 2D point"""
    x: float
    y: float
    


In [12]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
## generate json schema
pp.pprint(Point.json_schema())

{ '$schema': 'http://json-schema.org/draft-06/schema#',
  'description': 'a 2D point',
  'properties': {'x': {'type': 'number'}, 'y': {'type': 'number'}},
  'required': ['x', 'y'],
  'type': 'object'}


- https://docs.python.org/3/library/pprint.html

In [13]:
print(Point.json_schema())

{'type': 'object', 'required': ['x', 'y'], 'properties': {'x': {'type': 'number'}, 'y': {'type': 'number'}}, 'description': 'a 2D point', '$schema': 'http://json-schema.org/draft-06/schema#'}


In [14]:
## Deserialization
dct = {
    'x': 3.14,
    'y': 2.50
}
Point.from_dict(dct)

Point(x=3.14, y=2.5)

In [15]:
p = Point(1.111,2.222)
p.x

1.111

In [16]:
## __str__()
p

Point(x=1.111, y=2.222)

In [35]:
try:
    dct2 = {
        'x': 3.21,
        'y': 'y-ax'
    }
    p2 = Point.from_dict(dct2)
except Exception as e:
    print(e)

'y-ax' is not of type 'number'

Failed validating 'type' in schema['properties']['y']:
    {'type': 'number'}

On instance['y']:
    'y-ax'


In [18]:
### custome validator
from dataclasses_jsonschema import FieldEncoder
from typing import NewType, Optional

PhoneNumber = NewType('PhoneNumber', str)

class PhoneNumberField(FieldEncoder):
    @property
    def json_schema(self):
        return {
            'type': 'string',
            'pattern': r'^(\([0-9]{3}\))?[0-9]{3}-[0-9]{4}$',
            'maxLength': 12
        }

JsonSchemaMixin.register_field_encoders({PhoneNumber: PhoneNumberField()})

@dataclass
class Person(JsonSchemaMixin):
    name: str
    phone_number: Optional[PhoneNumber]


In [19]:
person1 = Person('John', '222-333-4444')
print(person1)

Person(name='John', phone_number='222-333-4444')


In [31]:
person2 = Person('Paul', '222-333-444AAA')
print(person2) ## validation is not available ## when convert from json, it is available
dir(person2)

Person(name='Paul', phone_number='222-333-444AAA')


['_JsonSchemaMixin__allow_additional_props',
 '_JsonSchemaMixin__compiled_schema',
 '_JsonSchemaMixin__decode_cache',
 '_JsonSchemaMixin__definitions',
 '_JsonSchemaMixin__discriminator_inherited',
 '_JsonSchemaMixin__discriminator_name',
 '_JsonSchemaMixin__encode_cache',
 '_JsonSchemaMixin__mapped_fields',
 '_JsonSchemaMixin__schema',
 '_JsonSchemaMixin__serialise_properties',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_decode_field',
 '_discriminator',
 '_encode_field',
 '_field_encoders',
 '_get_field_definitions',
 '_get_field_meta',
 '_get_field_schema',
 '_get_field_type_name',
 '_get_fi

In [34]:
try:
    person3 = Person('George') ## validation is not available
except Exception as e:
    print(e)

__init__() missing 1 required positional argument: 'phone_number'


In [24]:
Person.json_schema()

{'type': 'object',
 'required': ['name'],
 'properties': {'name': {'type': 'string'},
  'phone_number': {'type': 'string',
   'pattern': '^(\\([0-9]{3}\\))?[0-9]{3}-[0-9]{4}$',
   'maxLength': 12}},
 'description': 'Person(name: str, phone_number: Optional[PhoneNumber])',
 '$schema': 'http://json-schema.org/draft-06/schema#'}

In [30]:
dct4 = {
    'name': 'Ringo',
    'phone_number': '123-4567'
}
import json
json4 = json.dumps(dct4)
json4
person4 = Person.from_json(json4)
person4

Person(name='Ringo', phone_number='123-4567')

In [33]:
try:
    
    dct5 = {
        'name': 'Mary',
        'phone_number': '03-123-4567'
    }
    import json
    json5 = json.dumps(dct5)
    json5
    person5 = Person.from_json(json5)
    person5
except Exception as e:
    print(e)
    


'03-123-4567' does not match '^(\\([0-9]{3}\\))?[0-9]{3}-[0-9]{4}$'

Failed validating 'pattern' in schema['properties']['phone_number']:
    {'maxLength': 12,
     'pattern': '^(\\([0-9]{3}\\))?[0-9]{3}-[0-9]{4}$',
     'type': 'string'}

On instance['phone_number']:
    '03-123-4567'


- https://stackoverflow.com/questions/47719838/how-to-catch-all-exceptions-in-try-catch-block-python

## OOP in python 3
### Define Class in python
- Data Object: Value Object, Enumeration, POJO, DTO
- Interface
- Entity
- Repository: DAO

### Why We Use complex data structure.
- Validate
- Easy to undersand
- Layer of Architecture

- OOP is a method of structuring a program by bundling related properties and behaviors into indibidual object.
- Conceptually, Objects are like the components of a system.
- At each step of the assembly procedure a system component processes some objects, ultimately raw data into a finished outcome.
- An object contains data, like the raw or preprocessed infomation at each step on an procedure, and behavior, like the action each procedure component performs.

https://realpython.com/python3-object-oriented-programming/